# Advanced Tallies in OpenMC

In this tutorial, we'll learn about more advanced tally options.

In [ ]:
import openmc

A model that warrants advanced tallies is necessarily more complex than a pincell, so we're going to use the built-in PWR assembly model in OpenMC.

In [ ]:
model = openmc.examples.pwr_assembly()

In [ ]:
model.geometry.root_universe.plot(width=(22, 22), pixels=(600, 600))

In [ ]:
model.materials

In order to make some of the plots we'll generate in this session more intuitive, we'll remove the reflecting boundary condition for the upper `YPlane` and right `XPlane` in order to introduce some asymmetry.

In [ ]:
assembly_surfaces = model.geometry.get_all_surfaces()

for surf_id, surface in assembly_surfaces.items():
    if isinstance(surface, openmc.YPlane) and surface.y0 > 0.0:
        surface.boundary_type = 'vacuum'
    if isinstance(surface, openmc.XPlane) and surface.x0 > 0.0:
        surface.boundary_type = 'vacuum'

# Mesh Tallies

OpenMC can tally results onto regular, rectilinear, cylindrical, spherical, and unstructured meshes. Here we'll look at how to setup a regular mesh tally and visualize it for this assembly model. To do so, we need to create a mesh filter.

In [ ]:
lower_left, upper_right = model.geometry.bounding_box
print(lower_left, upper_right)

In [ ]:
mesh = openmc.RegularMesh()
mesh.lower_left = lower_left[:2]
mesh.upper_right = upper_right[:2]
mesh.dimension = (50, 50)
print(mesh)

mesh_filter = openmc.MeshFilter(mesh)

Learning from our last session on tallies, we'll include a tally with all of the scores needed for determining the neutron source.

In [ ]:
mesh_tally = openmc.Tally()
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['flux', 'heating']

With these tallies setup, we'll apply them and and run the model.

In [ ]:
model.tallies = [mesh_tally]

model.settings.particles = 1000
model.settings.batches = 15 
model.settings.inactive = 10

In [ ]:
statepoint = model.run()

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    mesh_tally_out = sp.get_tally(id=mesh_tally.id)

In [ ]:
mesh_flux = mesh_tally_out.get_values(scores=['flux'])
mesh_flux = mesh_flux.reshape(mesh.dimension)

In [ ]:
import matplotlib.pyplot as plt
img = plt.imshow(mesh_flux, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X [cm]')
plt.ylabel('Y [cm]')
plt.colorbar(img, label='Flux (unnormalized)')

Just like in our last tutorial, we need to renormalize these flux values by (i) multiplying by the neutron source rate and (ii) dividing by the volume of each tally bin in order to get into units of neutrons/cm$^2$/s.

In [ ]:
heating = mesh_tally_out.get_values(scores=['heating']).sum()

J_to_eV = 1 / 1.6e-19
power = 17.34e6
neutron_source = power * J_to_eV / heating
print(neutron_source)

For the volume normalization, we'll divide the flux values by the volume of a mesh voxel. Again, we're working with a 2D model so we'll assume an axial length of 1 cm.

In [ ]:
import numpy as np

volume = np.prod((mesh.upper_right - mesh.lower_left) / mesh.dimension)
print(volume)

In [ ]:
img = plt.imshow(mesh_flux * neutron_source / volume, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X [cm]')
plt.ylabel('Y [cm]')
plt.colorbar(img, label='Flux (unnormalized)')

In [ ]:
mesh_heat = neutron_source * mesh_tally_out.get_values(scores=['heating']) / volume * 1.602e-19
mesh_heat = mesh_heat.reshape(mesh.dimension)

In [ ]:
img = plt.imshow(mesh_heat, origin='lower', extent=[-10.71, 10.71, -10.71, 10.71])
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar(img, label='Heating (W/cm$^3$/s)')

### Manipulating the tally arrays

The `get_values()` method gives us an array with three dimensions: (filters, nuclides, scores). If you have multiple filters in a tally, the `get_reshaped_data()` method will give you a separate dimension for each filter. For our mesh case, this effectively gives the same thing as `get_values()` since there's only a single filter:

In [ ]:
mesh_tally_out.shape

In [ ]:
mesh_tally_out.get_reshaped_data().shape

However, there is also an `expand_dims` argument that will expand a mesh filter into multiple dimensions:

In [ ]:
mesh_data = mesh_tally_out.get_reshaped_data(expand_dims=True)
mesh_data.shape

In [ ]:
mesh_data = mesh_tally_out.get_reshaped_data(expand_dims=True).squeeze()
mesh_data.shape

Now we can index the array if we want to pull out specific values.

In [ ]:
mesh_data[0, 0]

However, this will still have our two different scores one after the other. It would be easier to postprocess this data if we could extract out each score one at a time. To do so, we'll use the `get_slice` method.

In [ ]:
flux_only = mesh_tally_out.get_slice(scores=['flux'])
flux_only.shape

In [ ]:
flux_reshaped = flux_only.get_reshaped_data(expand_dims=True).squeeze()
flux_reshaped.shape

In [ ]:
flux_reshaped[0, 0]

## Distributed cells (distribcells)

So this gives us a fairly good idea of what the flux and power distributions look like in this model, but we often want to know the per-pin power generation rate -- something that is hard to post-process with the tallies above (especially because the mesh tally is not conformal to the geometry). We can use a distribcell tally to produce this information easily.

A distributed cell (distribcell) is how OpenMC stores cells in universes which are repeated in lattices. In short, each cell in OpenMC is associated with an *id* and an *instance*. If a cell is repeated multiple times throughout a geometry, that cell has the same id, but with unique instances.

First, we'll want to create a distribcell tally for the cell containing our fuel material. Based on the list above, our fuel material has the name "Fuel". We'll use that to identify the cell we want to setup a distribcell tally for.

In [ ]:
fuel_cell = None

for cell_id, cell in model.geometry.get_all_material_cells().items():
    if cell.fill.name == 'Fuel':
        fuel_cell = cell
        
print(fuel_cell)

In [ ]:
model.geometry.determine_paths()
print(fuel_cell.num_instances)

In [ ]:
distribcell_filter = openmc.DistribcellFilter(fuel_cell)

In [ ]:
dcell_tally = openmc.Tally()
dcell_tally.filters = [distribcell_filter]
dcell_tally.scores = ['heating']

In [ ]:
model.tallies = [dcell_tally]

In [ ]:
statepoint = model.run()

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    dcell_tally_out = sp.tallies[dcell_tally.id]
    heat = dcell_tally_out.get_values(scores=['heating'])    

heat_df = dcell_tally_out.get_pandas_dataframe()
heat_df

As before, we'll create a normalized mean for the heating tally. A little inspection of the cell allows us to calculated the appropriate volume for the tally.

In [ ]:
print(fuel_cell.region)

In [ ]:
surfaces = model.geometry.get_all_surfaces()
fuel_cell_surf = surfaces[1]
print(fuel_cell_surf)

In [ ]:
volume = np.pi * fuel_cell_surf.r**2
print(volume)

In [ ]:
heat_df['power (W)'] = power * heat_df['mean'] / heating
heat_df['power (kW)'] = 1e-03 * heat_df['power (W)']

In [ ]:
pin_powers = heat_df[['level 2', 'level 3', 'distribcell', 'power (kW)']].copy()

In [ ]:
pin_powers